In [100]:
rawData = sc.textFile("../data/ML_spark/MovieLens/u.data")
print rawData.first()
rawRatings = rawData.map(lambda x: x.split('\t'))
rawRatings.take(5)

196	242	3	881250949


[[u'196', u'242', u'3', u'881250949'],
 [u'186', u'302', u'3', u'891717742'],
 [u'22', u'377', u'1', u'878887116'],
 [u'244', u'51', u'2', u'880606923'],
 [u'166', u'346', u'1', u'886397596']]

In [101]:
from pyspark.mllib.recommendation import Rating
from pyspark.mllib.recommendation import ALS
ratings = rawRatings.map(lambda x : Rating(int(x[0]),int(x[1]),float(x[2])))
print ratings.first()

Rating(user=196, product=242, rating=3.0)


In [107]:
model = ALS.train(ratings,50)
# modelImplicit = ALS.(ratings,50,alpha=0.02)
userFeatures = model.userFeatures()
print userFeatures.take(2)

[(1, array('d', [-0.13104072213172913, -0.282988965511322, 0.24604278802871704, -0.06333297491073608, -0.0932791605591774, 0.5691770911216736, -0.1593635380268097, -0.31947049498558044, 0.40703240036964417, 0.100776307284832, -0.07234027981758118, -0.13184002041816711, 0.09165998548269272, 0.03929411247372627, -0.5452826619148254, -0.6004565954208374, -0.4491104185581207, -0.3165319859981537, 0.13982048630714417, 0.6148051619529724, -0.19402047991752625, -0.2463371455669403, -0.42587268352508545, -0.4392813742160797, -0.27058282494544983, 0.4654891788959503, 0.09670299291610718, 0.690264880657196, -0.3652711510658264, 0.1276160180568695, -0.2946450412273407, -0.1003253310918808, 0.44784021377563477, 0.08422166109085083, 0.004761628340929747, 0.3710950016975403, 0.03529037535190582, -0.003700000699609518, 0.20448213815689087, -0.5644260048866272, 0.3054317533969879, -0.12391532212495804, -0.021917996928095818, -0.2748047411441803, 0.18297477066516876, -0.11456761509180069, 0.08473280072

In [105]:
predictRating = model.predict(789,123)
# predictRatingImplicit = modelImplicit.predict(789,123)
print predictRating
# print predictRatingImplicit

3.76599662082


In [5]:
userId = 789
K = 10


In [108]:
topKRecs = model.recommendProducts(userId,K)
for rec in topKRecs:
    print rec
moviesForUser = ratings.groupBy(lambda x : x.user).mapValues(list).lookup(userId)
# print moviesForUser
for i in moviesForUser[0][:10]:
    print i.product
# for 
# print moviesForUser

Rating(user=789, product=675, rating=6.367017598182441)
Rating(user=789, product=179, rating=5.6896459072899335)
Rating(user=789, product=514, rating=5.457768288024388)
Rating(user=789, product=101, rating=5.433179519202287)
Rating(user=789, product=653, rating=5.394240790943933)
Rating(user=789, product=573, rating=5.361399367897831)
Rating(user=789, product=182, rating=5.266709390311053)
Rating(user=789, product=183, rating=5.2116836182726045)
Rating(user=789, product=135, rating=5.203360119834471)
Rating(user=789, product=611, rating=5.1741158219632695)
1012
127
475
93
1161
286
293
9
50
294


In [7]:
movies = sc.textFile("../data/ML_spark/MovieLens/u.item")
titles = movies.map(lambda line: (int(line.split('|')[0]),line.split('|')[1])).collectAsMap()
print titles

{1: u'Toy Story (1995)', 2: u'GoldenEye (1995)', 3: u'Four Rooms (1995)', 4: u'Get Shorty (1995)', 5: u'Copycat (1995)', 6: u'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)', 7: u'Twelve Monkeys (1995)', 8: u'Babe (1995)', 9: u'Dead Man Walking (1995)', 10: u'Richard III (1995)', 11: u'Seven (Se7en) (1995)', 12: u'Usual Suspects, The (1995)', 13: u'Mighty Aphrodite (1995)', 14: u'Postino, Il (1994)', 15: u"Mr. Holland's Opus (1995)", 16: u'French Twist (Gazon maudit) (1995)', 17: u'From Dusk Till Dawn (1996)', 18: u'White Balloon, The (1995)', 19: u"Antonia's Line (1995)", 20: u'Angels and Insects (1995)', 21: u'Muppet Treasure Island (1996)', 22: u'Braveheart (1995)', 23: u'Taxi Driver (1976)', 24: u'Rumble in the Bronx (1995)', 25: u'Birdcage, The (1996)', 26: u'Brothers McMullen, The (1995)', 27: u'Bad Boys (1995)', 28: u'Apollo 13 (1995)', 29: u'Batman Forever (1995)', 30: u'Belle de jour (1967)', 31: u'Crimson Tide (1995)', 32: u'Crumb (1994)', 33: u'Desperado (1995)', 34: u

In [8]:
for i,rec in enumerate(topKRecs):
    print 'rank:'+str(i)+' '+str(titles[rec.product])+':'+str(rec.rating)

rank:0 Eat Drink Man Woman (1994):6.15284491245
rank:1 Casino (1995):6.10163520889
rank:2 Sunset Blvd. (1950):5.62902786188
rank:3 Once Upon a Time in the West (1969):5.6082416842
rank:4 Belle de jour (1967):5.54629169828
rank:5 Wrong Trousers, The (1993):5.50376600967
rank:6 Jean de Florette (1986):5.45471934496
rank:7 Searching for Bobby Fischer (1993):5.36556722981
rank:8 Lawrence of Arabia (1962):5.30207731145
rank:9 Nikita (La Femme Nikita) (1990):5.28293214622


In [9]:
import numpy as np
def cosineSImilarity(x,y):
    return np.dot(x,y)/(np.linalg.norm(x)*np.linalg.norm(y))

In [10]:
testx = np.array([1.0,2.0,3.0])
print cosineSImilarity(testx,testx)
print np.linalg.norm(testx)

1.0
3.74165738677


In [11]:
itemId = 567
itemFactor = model.productFeatures().lookup(itemId)[0]
# itemFactor = itemFactor[1]
print itemFactor
# model.productFeatures().collect()
sims = model.productFeatures().map(lambda (id,factor):(id,cosineSImilarity(np.array(factor),np.array(itemFactor))))

array('d', [0.49078404903411865, 0.012953451834619045, 0.7479921579360962, 0.4946943521499634, 0.6509256958961487, -0.4265731871128082, -0.13153676688671112, -0.3975328505039215, -0.38279953598976135, -1.2912747859954834, 0.2635449469089508, -0.8708798289299011, 0.6441885828971863, 0.33515551686286926, 0.14537470042705536, 0.9009832143783569, 0.009731626138091087, 0.023789778351783752, 0.27959856390953064, 0.007528654765337706, 0.08425039052963257, 0.43897491693496704, -0.16026359796524048, -0.20233289897441864, 2.104543924331665, 0.10034932941198349, -0.3649766743183136, 0.39258718490600586, 1.0704870223999023, -0.5270863175392151, 0.9657369256019592, -0.5661191344261169, -0.9723479151725769, -0.18243126571178436, 0.4041025638580322, -0.6465684175491333, 0.196022629737854, -0.6070469617843628, -0.08054587990045547, -0.14497430622577667, 0.7804021835327148, 0.03698401153087616, 0.12916600704193115, -0.6674327254295349, 0.9700697064399719, 0.42157983779907227, 0.7554676532745361, -1.258

In [12]:
# sortedSims = sims.sortBy(lambda (x,y) :y).collect()
sims.sortBy(lambda (x,y):y,ascending=False).take(10)

[(567, 1.0),
 (719, 0.71248410792997596),
 (413, 0.69059376096633496),
 (25, 0.67763829324023017),
 (1314, 0.67407970230964287),
 (1376, 0.67392156181875307),
 (985, 0.67356021728588122),
 (302, 0.67100724629582231),
 (1416, 0.66989727830701273),
 (853, 0.65847813087617835)]

In [13]:
actualRating = moviesForUser[0][0]
print actualRating

Rating(user=789, product=1012, rating=4.0)


In [14]:
predictedRating = model.predict(789,actualRating.product)
import math
print predictedRating
print math.pow(predictedRating-actualRating.rating,2)

4.05457508298
0.00297843968204


In [15]:
userProducts = ratings.map(lambda rating:(rating.user,rating.product))
# model_broadcast = sc.broadcast(model)
print userProducts.take(1)[0]
# predictions = userProducts.map(lambda (x,y):(x,y,model_broadcast.predict(x,y)))
predictions = model.predictAll(userProducts).map(lambda rating:((rating.user,rating.product),rating.rating))
print predictions.take(5)

(196, 242)
[((195, 1084), 3.9097299400338623), ((58, 1084), 4.127441864829657), ((541, 1084), 4.0361277777635545), ((470, 1084), 3.050509557952725), ((682, 1084), 1.974494399379628)]


In [16]:
ratingsAndPredictions = ratings.map(lambda rating:((rating.user,rating.product),rating.rating)).join(predictions)
print ratingsAndPredictions.take(5)

[((621, 577), (3.0, 3.024912394805944)), ((363, 849), (2.0, 1.9396191866428834)), ((109, 365), (4.0, 3.873292944661129)), ((254, 622), (4.0, 3.9909069692632793)), ((720, 286), (5.0, 4.966204197437744))]


In [17]:
MSE = ratingsAndPredictions.map(lambda ((x,y),(m,n)):math.pow(m-n,2)).reduce(lambda x,y:x+y)/ratingsAndPredictions.count()
print MSE
print math.sqrt(MSE)

0.110573849112
0.332526463777


In [96]:
# APK
def avgPrecisionK(actual, predicted,k=10):
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0
    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 1.0

    return score / min(len(actual), k)

In [63]:
actualMovies = [rating.product for rating in moviesForUser[0]]
predictMovies = [rating.product for rating in topKRecs]
print actualMovies
print predictMovies

[1012, 127, 475, 93, 1161, 286, 293, 9, 50, 294, 181, 1, 1008, 508, 284, 1017, 137, 111, 742, 248, 249, 1007, 591, 150, 276, 151, 129, 100, 741, 288, 762, 628, 124]
[45, 693, 488, 646, 30, 169, 165, 87, 511, 198]


In [64]:
apk10 = avgPrecisionK(actualMovies,predictMovies,10)
print apk10

0.0


In [65]:
itemFactors = model.productFeatures().map(lambda (id,factor):factor).collect()
itemMatrix = np.array(itemFactors)
print itemMatrix
print itemMatrix.shape

[[-0.56040549  0.12426949  1.39385641 ..., -2.10716724  0.93263996
  -0.06961832]
 [-0.6338225  -0.45199269  0.29880574 ..., -1.60107303  0.55651504
  -0.30998191]
 [ 0.27567902 -0.55576426  1.56970084 ..., -1.82570267  1.02763498
   0.37729299]
 ..., 
 [ 0.05657623 -0.05834675  0.12872325 ..., -0.20585941  0.08904345
   0.14201939]
 [ 0.02720444 -0.03196245  0.1858432  ..., -0.18072413 -0.32529363
   0.22977404]
 [ 0.14529222 -0.20424826  0.36027607 ..., -0.46585017  0.47996596
  -0.0286847 ]]
(1682, 50)


In [66]:
# 便于后面进行map计算的时候使所有map都有效，这里在实际当中的意思是把数据广播到任何一个节点
imBroadcast = sc.broadcast(itemMatrix)

In [67]:
userVector = model.userFeatures().map(lambda (userId,array):(userId,np.array(array)))
# print userVector[0]
userVector = userVector.map(lambda (userId,x): (userId,imBroadcast.value.dot((np.array(x).transpose()))))
userVectorId = userVector.map(lambda (userId,x) : (userId,[(xx,i) for i,xx in enumerate(x.tolist())]))
sortUserVectorId = userVectorId.map(lambda (userId,x):(userId,sorted(x,key=lambda x:x[0],reverse=True)))
sortUserVectorRecId = sortUserVectorId.map(lambda (userId,x): (userId,[xx[1] for xx in x]))
# print sortUserVectorRecId.take(2)
sortUserVectorRecId.count()

943

In [98]:
#val userMovies = ratings.map{ case Rating(user, product, rating) =>(user, product) }.groupBy(_._1)

userMovies = ratings.map(lambda rating: (rating.user,rating.product)).groupBy(lambda (x,y):x)
userMovies = userMovies.map(lambda (userId,x):(userId, [xx[1] for xx in x] ))
# userMovies.take(1)
# allAPK=sortUserVectorRecId.join(userMovies).map(lambda (userId,(predicted, actual)):(userId,avgPrecisionK(actual,predicted,10)))
# print allAPK.map(lambda (x,y):y).reduce(lambda x,y:x+y)/sortUserVectorRecId.count()
allAPK=sortUserVectorRecId.join(userMovies).map(lambda (userId,(predicted, actual)):avgPrecisionK(actual,predicted,2000))
# print allAPK.take(10)
print allAPK.reduce(lambda x,y:x+y)/allAPK.count()

0.115484271925


In [69]:
# 用MLlib内置的evaluation函数
from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.mllib.evaluation import RankingMetrics
predictedAndTrue = ratingsAndPredictions.map(lambda ((userId,product),(predicted, actual)):(predicted,actual))
print predictedAndTrue.take(1)
regressionMetrics = RegressionMetrics(predictedAndTrue)
print "Mean Squared Error = %f"%regressionMetrics.meanSquaredError
print "Root Mean Squared Error %f"% regressionMetrics.rootMeanSquaredError

[(3.0, 3.024912394805944)]
Mean Squared Error = 0.110574
Root Mean Squared Error 0.332526


In [99]:
#MAP
# The implementation of the average precision at the K function in RankingMetrics is slightly different from ours, 
# so we will get different results. However, the computation of the overall mean average precision (MAP, which does 
# not use a threshold at K) is the same as our function if we select K to be very high (say, at least as high as the number of items in our item set)
sortedLabels = sortUserVectorRecId.join(userMovies).map(lambda (userId,(predicted, actual)):(predicted,actual))
# print sortedLabels.take(1)
rankMetrics = RankingMetrics(sortedLabels)
print "Mean Average Precision = %f" % rankMetrics.meanAveragePrecision
print "Mean Average Precision(at K=10) = %f" % rankMetrics.precisionAt(5)

Mean Average Precision = 0.115484
Mean Average Precision(at K=10) = 0.131707
